In [1]:
from collections import defaultdict
from functools import lru_cache
import random
import requests
from requests import Session

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statbotics

%matplotlib notebook

sb = statbotics.Statbotics()

In [2]:
all_teams = sb.get_team_years(year=2023, limit=10000)
all_teams_dict = {t["team"]: t for t in all_teams}
len(all_teams)

3286

In [8]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

def get_tba(url):
    response = session.get(read_prefix + url)
    return response.json()

@lru_cache()
def get_event_teams(key):
    data = get_tba(f"event/{key}/teams")
    return [int(x["key"][3:]) for x in data]

@lru_cache()
def get_event_matches(key):
    data = get_tba(f"event/{key}/matches")
    return [{"red": [int(y[3:]) for y in x["alliances"]["red"]["team_keys"]], "blue": [int(y[3:]) for y in x["alliances"]["blue"]["team_keys"]]} for x in data]

@lru_cache()
def get_event_rankings(key):
    data = get_tba(f"event/{key}/rankings")
    return [int(x["team_key"][3:]) for x in data["rankings"]]


In [9]:
divisions = [
    get_event_teams("2023arc"),
    get_event_teams("2023cur"),
    get_event_teams("2023dal"),
    get_event_teams("2023gal"),
    get_event_teams("2023hop"),
    get_event_teams("2023joh"),
    get_event_teams("2023mil"),
    get_event_teams("2023new"),
]

champ_team_nums = []
for division in divisions:
    champ_team_nums.extend(division)

division_matches = [
    get_event_matches("2023arc"),
    get_event_matches("2023cur"),
    get_event_matches("2023dal"),
    get_event_matches("2023gal"),
    get_event_matches("2023hop"),
    get_event_matches("2023joh"),
    get_event_matches("2023mil"),
    get_event_matches("2023new"),
]

division_ranks = [
    get_event_rankings("2023arc"),
    get_event_rankings("2023cur"),
    get_event_rankings("2023dal"),
    get_event_rankings("2023gal"),
    get_event_rankings("2023hop"),
    get_event_rankings("2023joh"),
    get_event_rankings("2023mil"),
    get_event_rankings("2023new"),
]

In [15]:
team_to_pre_champs_epa = {t: all_teams_dict[t]["epa_pre_champs"] for t in champ_team_nums}
team_to_end_epa = {t: all_teams_dict[t]["epa_end"] for t in champ_team_nums}
team_to_rp_1_epa = {t: all_teams_dict[t]["rp_1_epa_end"] for t in champ_team_nums}
team_to_rp_2_epa = {t: all_teams_dict[t]["rp_2_epa_end"] for t in champ_team_nums}

print(team_to_epa[2056], team_to_rp_1_epa[2056], team_to_rp_2_epa[2056])
print(team_to_epa[254], team_to_rp_1_epa[254], team_to_rp_2_epa[254])

83.98 1.029 0.6023
80.14 0.9967 0.4425


In [16]:
TOTAL_MEAN = 74.57
TOTAL_SD = 29.36

In [17]:
@lru_cache
def make_request(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.text
    lines = data.split("\n")
    return lines

@lru_cache
def get_schedule(num_teams: int, num_matches: int):
    # TODO: remove this once we have pre-generated schedules for 100+ teams
    if num_teams > 100:
        schedule1 = get_schedule(100, num_matches)
        schedule2 = get_schedule(num_teams - 100, num_matches)
        schedule2 = [
            {"red": [team + 100 for team in match["red"]], "blue": [team + 100 for team in match["blue"]]}
            for match in schedule2
        ]
        return schedule1 + schedule2

    # load csv from external URL using requests
    lines = make_request(f"https://raw.githubusercontent.com/Team254/cheesy-arena/main/schedules/{num_teams}_{num_matches}.csv")
    
    schedule = []
    for line in lines:
        match = line.split(",")
        if len(match) < 12:
            continue
        red = [int(match[0]), int(match[2]), int(match[4])]
        blue = [int(match[6]), int(match[8]), int(match[10])]
        schedule.append({"red": red, "blue": blue})
        
    return schedule

In [18]:
def get_win_prob(a, b):
    return 1 / (1 + 10 ** (((-5 / 8) * (a - b)) / TOTAL_SD))

def get_rp_pred(a):
    return 1 / (1 + np.e ** (-4 * (a - 0.5)))

def sim_single_quals(teams, schedule, team_to_epa, team_to_rp_1_epa, team_to_rp_2_epa):
    curr_sim_matches = {t: 0 for t in teams}
    curr_sim_rps = {t: 0 for t in teams}
    for m in schedule:
        red_epa = sum(team_to_epa[x] for x in m["red"])
        blue_epa = sum(team_to_epa[x] for x in m["blue"])
        red_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["red"])
        blue_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["blue"])
        red_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["red"])
        blue_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["blue"])
        
        win_prob = get_win_prob(red_epa, blue_epa)
        red_win = 1 if random.random() < win_prob else 0

        red_rp_1_prob = get_rp_pred(0.9 * red_rp_1_epa)
        red_rp_1 = 1 if random.random() < red_rp_1_prob else 0
        
        red_rp_2_prob = get_rp_pred(red_rp_2_epa)
        red_rp_2 = 1 if random.random() < red_rp_2_prob else 0
        
        blue_rp_1_prob = get_rp_pred(0.9 * blue_rp_1_epa)
        blue_rp_1 = 1 if random.random() < blue_rp_1_prob else 0
        
        blue_rp_2_prob = get_rp_pred(blue_rp_2_epa)
        blue_rp_2 = 1 if random.random() < blue_rp_2_prob else 0

        red_rps = red_rp_1 + red_rp_2 + (2 if red_win else 0)
        blue_rps = blue_rp_1 + blue_rp_2 + (0 if red_win else 2)
        
        for x in m["red"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += red_rps
            
        for x in m["blue"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += blue_rps
            
    curr_sim_ranks = sorted(curr_sim_rps.items(), key=lambda x: [-x[1], random.random()])
    
    return [x[0] for x in curr_sim_ranks]

def softmax_select(options, mult=1):
    exp = [np.e ** (mult * o) for o in options]
    sum_exp = sum(exp)
    exp = [e / sum_exp for e in exp]
    rand = random.random()
    curr, i = 0, 0
    while curr < rand:
        curr += exp[i]
        i += 1
    return i - 1

def sim_alliance_selection(ranks, team_to_epa):
    alliances = [[], [], [], [], [], [], [], []]
    locked_teams = []
    remaining_teams = [(r, team_to_epa[r]) for r in ranks]
    
    r1_mult = 1 / 3
    r2_mult = 1 / 2
    
    def handle_selection(selector, reject=True):
        selected = None
        while selected is None:
            temp_remaining_teams = [r for r in remaining_teams if r not in locked_teams]
            _selected = softmax_select([r[1] for r in temp_remaining_teams], r1_mult)
            _selected_team = temp_remaining_teams[_selected]
            orig_rank = ranks.index(_selected_team[0]) + 1
            if not reject or orig_rank > 8:
                selected = _selected
                continue
                
            selected_rank = remaining_teams.index(_selected_team) + 1
            remaining_best_epas = sorted(temp_remaining_teams[_selected + 1:], key=lambda x: -x[1])
            if remaining_best_epas[selected_rank][1] > selector[1] + 5:
                locked_teams.append(_selected_team)
            else:
                selected = _selected
                
        return remaining_teams.index(temp_remaining_teams[selected])
    
    # Captain and Round 1
    alliances[0].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[0].append(remaining_teams.pop(selected))
    
    alliances[1].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[1].append(remaining_teams.pop(selected))
    
    alliances[2].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[2].append(remaining_teams.pop(selected))
    
    alliances[3].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[3].append(remaining_teams.pop(selected))

    alliances[4].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[4].append(remaining_teams.pop(selected))
    
    alliances[5].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[5].append(remaining_teams.pop(selected))
    
    alliances[6].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[6].append(remaining_teams.pop(selected))
    
    alliances[7].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[7].append(remaining_teams.pop(selected))
    
    # Round 2
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    
    # Round 3
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    
    aepas = [sum(a[1] for a in alliance[:3]) for alliance in alliances]
    return [[a[0] for a in alliance] for alliance in alliances], aepas

def sim_single_elims(alliances, team_to_epa):
    scores, grids = [], []
    
    aepas = [sum(team_to_epa[a] for a in alliance[:3]) for alliance in alliances]
    ms = [[0, 7], [3, 4], [1, 6], [2, 5], [], [], [], [], [], [], [], [], []]
    
    def _get_win_prob(i):
        return get_win_prob(aepas[ms[i][0]], aepas[ms[i][1]])

    m1_red_winner = random.random() < _get_win_prob(0)
    ms[4].append(ms[0][1] if m1_red_winner else ms[0][0])
    ms[6].append(ms[0][0] if m1_red_winner else ms[0][1])
    
    m2_red_winner = random.random() < _get_win_prob(1)
    ms[4].append(ms[1][1] if m2_red_winner else ms[1][0])
    ms[6].append(ms[1][0] if m2_red_winner else ms[1][1])
    
    m3_red_winner = random.random() < _get_win_prob(2)
    ms[5].append(ms[2][1] if m3_red_winner else ms[2][0])
    ms[7].append(ms[2][0] if m3_red_winner else ms[2][1])
    
    m4_red_winner = random.random() < _get_win_prob(3)
    ms[5].append(ms[3][1] if m4_red_winner else ms[3][0])
    ms[7].append(ms[3][0] if m4_red_winner else ms[3][1])
    
    m5_red_winner = random.random() < _get_win_prob(4)
    ms[9].append(ms[4][0] if m5_red_winner else ms[4][1])
    
    m6_red_winner = random.random() < _get_win_prob(5)
    ms[8].append(ms[5][0] if m6_red_winner else ms[5][1])
    
    m7_red_winner = random.random() < _get_win_prob(6)
    ms[8].append(ms[6][1] if m7_red_winner else ms[6][0])
    ms[10].append(ms[6][0] if m7_red_winner else ms[6][1])
    
    m8_red_winner = random.random() < _get_win_prob(7)
    ms[9].append(ms[7][1] if m8_red_winner else ms[7][0])
    ms[10].append(ms[7][0] if m8_red_winner else ms[7][1])
    
    m9_red_winner = random.random() < _get_win_prob(8)
    ms[11].append(ms[8][0] if m9_red_winner else ms[8][1])
    
    m10_red_winner = random.random() < _get_win_prob(9)
    ms[11].append(ms[9][0] if m10_red_winner else ms[9][1])
    
    m11_red_winner = random.random() < _get_win_prob(10)
    ms[12].append(ms[10][1] if m11_red_winner else ms[10][0])
    finalist_1 = ms[10][0] if m11_red_winner else ms[10][1]
    
    m12_red_winner = random.random() < _get_win_prob(11)
    fourth_place = ms[11][1] if m12_red_winner else ms[11][0]
    ms[12].append(ms[11][0] if m12_red_winner else ms[11][1])
    
    m13_red_winner = random.random() < _get_win_prob(12)
    third_place = ms[12][1] if m13_red_winner else ms[12][0]
    finalist_2 = ms[12][0] if m13_red_winner else ms[12][1]
    
    
    f1_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f2_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f3_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    winner = finalist_1 if f1_red_winner + f2_red_winner + f3_red_winner >= 2 else finalist_2
    second_place = finalist_2 if winner == finalist_1 else finalist_1
    
    return winner, second_place, third_place, fourth_place


In [26]:
# Options: Pre Divisions, Pre Schedules, Final, Post Quals
def run_simulation(status="Pre Divisions", num_sims=1000):
    overall_winner_count = defaultdict(int)
    overall_finalist_count = defaultdict(int)
    overall_third_place_count = defaultdict(int)
    overall_fourth_place_count = defaultdict(int)

    winner_count = defaultdict(int)
    finalist_count = defaultdict(int)
    third_place_count = defaultdict(int)
    fourth_place_count = defaultdict(int)

    winner_alliances = defaultdict(int)
    winner_best_epas = []
    alliance_epas = []
    best_epas = []
    winner_epas = []

    einstein_winner_best_epas = []
    einstein_alliance_epas = []
    einstein_best_epas = []
    einstein_winner_epas = []

    einstein_winner = defaultdict(int)
    einstein_finalist = defaultdict(int)
    einstein_third = defaultdict(int)
    einstein_fourth = defaultdict(int)

    einstein_pairs = defaultdict(int)
    winning_pairs = defaultdict(int)
    pairs = defaultdict(int)

    qual_rank_list = defaultdict(lambda: [0] * 78)

    for sim in range(num_sims):
        if sim % 100 == 0:
            print(sim)

        noise = 4.5
        if status == "Post Quals":
            curr_team_to_epa = {k: v + np.random.normal(0, noise) for k, v in team_to_end_epa.items()}
        else:
            curr_team_to_epa = {k : v + np.random.normal(0, noise) for k, v in team_to_pre_champs_epa.items()}

        rp_noise = 0.1
        curr_team_to_rp_1_epa = {k : v + np.random.normal(0, rp_noise) for k, v in team_to_rp_1_epa.items()}
        curr_team_to_rp_2_epa = {k : v + np.random.normal(0, rp_noise) for k, v in team_to_rp_2_epa.items()}

        einstein_alliances = []
        einstein_aepas = []
        
        if status == "Pre Divisions":
            random.shuffle(champ_team_nums)
            division_1 = champ_team_nums[:78]
            division_2 = champ_team_nums[78:156]
            division_3 = champ_team_nums[156:233]
            division_4 = champ_team_nums[233:310]
            division_5 = champ_team_nums[310:388]
            division_6 = champ_team_nums[388:466]
            division_7 = champ_team_nums[466:543]
            division_8 = champ_team_nums[543:620]
            curr_divisions = [division_1, division_2, division_3, division_4, division_5, division_6, division_7, division_8]
        else:
            curr_divisions = divisions
        
        for division, matches, ranks in zip(curr_divisions, division_matches, division_ranks):
            random.shuffle(division)
            
            if status == "Final":
                schedule = matches
            else:
                schedule = get_schedule(len(division), 10)
                schedule = [
                    {
                        "red": [division[x - 1] for x in m["red"]],
                        "blue": [division[x - 1] for x in m["blue"]],
                    }
                    for m in schedule
                ]
                
            if status != "Post Quals":
                qual_ranks = sim_single_quals(division, schedule, curr_team_to_epa, curr_team_to_rp_1_epa, curr_team_to_rp_2_epa)
            else:
                qual_ranks = ranks
            
            for i, t in enumerate(qual_ranks):
                qual_rank_list[t][i] += 1

            alliances, aepas = sim_alliance_selection(qual_ranks, curr_team_to_epa)
            for alliance in alliances:
                pairs[frozenset((alliance[0], alliance[1]))] += 1
            
            winner, finalist, third_place, fourth_place = sim_single_elims(alliances, curr_team_to_epa)
            einstein_alliances.append(alliances[winner])
            einstein_aepas.append(aepas[winner])

            einstein_pairs[frozenset((alliances[winner][0], alliances[winner][1]))] += 1

            winner_alliances[winner] += 1
            best_epa = max(aepas)
            winner_epa = aepas[winner]
            winner_best_epas.append(best_epa == winner_epa)
            alliance_epas.extend(aepas)
            best_epas.append(best_epa)
            winner_epas.append(winner_epa)

            for t in alliances[winner]:
                winner_count[t] += 1

            for t in alliances[finalist]:
                finalist_count[t] += 1

            for t in alliances[third_place]:
                third_place_count[t] += 1

            for t in alliances[fourth_place]:
                fourth_place_count[t] += 1

        winner, finalist, third_place, fourth_place = sim_single_elims(einstein_alliances, curr_team_to_epa)
        einstein_winner[winner] += 1
        einstein_finalist[finalist] += 1
        einstein_third[third_place] += 1
        einstein_fourth[fourth_place] += 1

        winning_pairs[frozenset((einstein_alliances[winner][0], einstein_alliances[winner][1]))] += 1

        einstein_best_epa = max(einstein_aepas)
        einstein_winner_epa = einstein_aepas[winner]
        einstein_winner_best_epas.append(einstein_best_epa == einstein_winner_epa)
        einstein_alliance_epas.extend(einstein_aepas)
        einstein_best_epas.append(einstein_best_epa)
        einstein_winner_epas.append(einstein_winner_epa)

        for t in einstein_alliances[winner]:
            overall_winner_count[t] += 1

        for t in einstein_alliances[finalist]:
            overall_finalist_count[t] += 1

        for t in einstein_alliances[third_place]:
            overall_third_place_count[t] += 1

        for t in einstein_alliances[fourth_place]:
            overall_fourth_place_count[t] += 1
            
    avg_ranks = {t: sum(((i + 1) * qual_rank_list[t][i]) for i in range(78)) / num_sims for t in qual_rank_list}
    top_1 = {t: sum(qual_rank_list[t][i] for i in range(1)) / num_sims for t in qual_rank_list}
    top_4 = {t: sum(qual_rank_list[t][i] for i in range(4)) / num_sims for t in qual_rank_list}
    top_8 = {t: sum(qual_rank_list[t][i] for i in range(8)) / num_sims for t in qual_rank_list}
    top_16 = {t: sum(qual_rank_list[t][i] for i in range(16)) / num_sims for t in qual_rank_list}
            
    return {
        "overall_winner_count": overall_winner_count,
        "overall_finalist_count": overall_finalist_count,
        "overall_third_place_count": overall_third_place_count,
        "overall_fourth_place_count": overall_fourth_place_count,
        "winner_count": winner_count,
        "finalist_count": finalist_count,
        "third_place_count": third_place_count,
        "fourth_place_count": fourth_place_count,
        "winner_alliances": winner_alliances,
        "winner_best_epas": winner_best_epas,
        "alliance_epas": alliance_epas,
        "best_epas": best_epas,
        "winner_epas": winner_epas,
        "einstein_winner_best_epas": einstein_winner_best_epas,
        "einstein_alliance_epas": einstein_alliance_epas,
        "einstein_best_epas": einstein_best_epas,
        "einstein_winner_epas": einstein_winner_epas,
        "einstein_winner": einstein_winner,
        "einstein_finalist": einstein_finalist,
        "einstein_third": einstein_third,
        "einstein_fourth": einstein_fourth,
        "pairs": pairs,
        "einstein_pairs": einstein_pairs,
        "winning_pairs": winning_pairs,
        "qual_rank_list": qual_rank_list,
        "avg_ranks": avg_ranks,
        "top_1": top_1,
        "top_4": top_4,
        "top_8": top_8,
        "top_16": top_16,
    }

In [30]:
post_quals = run_simulation("Post Quals", 100000)
final = run_simulation("Final", 100000)
after = run_simulation("Pre Schedule", 100000)
before = run_simulation("Pre Divisions", 100000)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100
43200
43300
43400
43500
43600
43700
43800
43900
44000
44100
44200
44300
44400
44500
44600
44700
44800
44900
45000
45100
45200
45300
45400
45500
45600
45700
45800
45900
46000
46100
46200
46300
46400
46500
46600
46700
46800
46900
47000
47100
47200
47300
47400
47500
47600
47700
47800
47900
48000
48100
48200
48300
48400
48500
48600
48700
48800
48900
49000
49100
49200
49300
49400
49500
49600
49700
49800
49900
50000
50100
50200
50300
50400
50500
50600
50700
50800
50900
51000
51100
51200
51300
51400
51500
51600
51700
51800
51900
52000
52100
52200
52300
52400
52500
52600
52700
52800
52900
53000
53100
53200
53300
53400
53500
53600
53700
53800
53900
54000
54100
54200
54300
54400
54500
54600
54700
54800
54900
55000
55100
55200
55300
55400
55500
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
5690

78800
78900
79000
79100
79200
79300
79400
79500
79600
79700
79800
79900
80000
80100
80200
80300
80400
80500
80600
80700
80800
80900
81000
81100
81200
81300
81400
81500
81600
81700
81800
81900
82000
82100
82200
82300
82400
82500
82600
82700
82800
82900
83000
83100
83200
83300
83400
83500
83600
83700
83800
83900
84000
84100
84200
84300
84400
84500
84600
84700
84800
84900
85000
85100
85200
85300
85400
85500
85600
85700
85800
85900
86000
86100
86200
86300
86400
86500
86600
86700
86800
86900
87000
87100
87200
87300
87400
87500
87600
87700
87800
87900
88000
88100
88200
88300
88400
88500
88600
88700
88800
88900
89000
89100
89200
89300
89400
89500
89600
89700
89800
89900
90000
90100
90200
90300
90400
90500
90600
90700
90800
90900
91000
91100
91200
91300
91400
91500
91600
91700
91800
91900
92000
92100
92200
92300
92400
92500
92600
92700
92800
92900
93000
93100
93200
93300
93400
93500
93600
93700
93800
93900
94000
94100
94200
94300
94400
94500
94600
94700
94800
94900
95000
95100
95200
95300
9540

In [31]:
num_sims = 100000

In [32]:
def pad(s, length = 15):
    return s + "&nbsp;" * (length - len(s))

def percent(x):
    return str(round(100 * x, 2)) + "%"

division_names = ["Archimedes", "Curie", "Daly", "Galileo", "Hopper", "Johnson", "Milstein", "Newton"]
division_to_team = {}
for i, division in enumerate(divisions):
    for t in division:
        division_to_team[t] = division_names[i]

print("<h2>Chance of winning Einstein</h2>")
print()
        
print("| Division | ", pad("Pre-Divisions Winner"), " | ", pad("Pre-Schedule Winner"), " | ", pad("Post-Schedule Winner"), " | ", pad("Post-Quals Winner"), " |")
print("| --- | :-: | :-: | :-: | :-: |")
for i in range(8):
    print("|", "|".join([division_names[i], percent(before["einstein_winner"][i] / num_sims), percent(after["einstein_winner"][i] / num_sims), percent(final["einstein_winner"][i] / num_sims), percent(post_quals["einstein_winner"][i] / num_sims)]) + "|")
print()
print()

def print_ranks(filter=lambda x: True, limit=25):
    print("| Rank | ", pad("Team"), " | ", pad("Division"), " | ", pad("Pre-Divisions Mean Rank"), " | ", pad("Pre-Schedule Mean Rank"), " | ", pad("Post-Schedule Mean Rank"), " | ", pad("Post-Quals Mean Rank"), " |")
    print("| --- | --- | --- | :-: | :-: | :-: | :-: |")
    for i, (key, _) in enumerate(sorted(post_quals["avg_ranks"].items(), key=lambda x: (filter(x[0]), x[1]))[:limit]):
        print("|", "|".join([str(i + 1), str(key), division_to_team[key], str(round(before["avg_ranks"][key], 2)), str(round(after["avg_ranks"][key], 2)), str(round(final["avg_ranks"][key], 2)), str(round(post_quals["avg_ranks"][key], 2))]), "|")
    print()
    
print("<h2>Chance of Ranking High</h2>")
print()
    
print_ranks()

print()
print("See Statbotics event pages for individual event simulations")
print()


def print_divisions(filter=lambda x: True, limit=25):
    print("| Rank | ", pad("Team"), " | ", pad("Division"), " | ", pad("Pre-Divisions Winner"), " | ", pad("Pre-Schedule Winner"), " | ", pad("Post-Schedule Winner"), " | ", pad("Post-Quals Winner"), " |")
    print("| ---- | ---- | --- | :-: | :-: | :-: | :-: |")
    for i, (key, _) in enumerate(sorted(post_quals["winner_count"].items(), key=lambda x: (filter(x[0]), -x[1]))[:limit]):
        print("|", "|".join([str(i + 1), str(key), division_to_team[key], percent(before["winner_count"][key] / num_sims), percent(after["winner_count"][key] / num_sims), percent(final["winner_count"][key] / num_sims), percent(post_quals["winner_count"][key] / num_sims)]) + "|")
    print()

print("<h2>Chance of winning division</h2>")
print()
    
print_divisions()

print("[details='By Division']")

for i in range(8):
    print("<h4>" + division_names[i] + "</h4>")
    print()
    print_divisions(filter=lambda x: 0 if x in divisions[i] else 1, limit=10)
    
print("[/details]")
print()
    
def print_einstein(filter=lambda x: True, limit=25):
    print("| Rank | ", pad("Team"), " | ", pad("Division"), " | ", pad("Pre-Divisions Winner"), " | ", pad("Pre-Schedule Winner"), " | ", pad("Post-Schedule Winner"), " | ", pad("Post-Quals Winner"), " |")
    print("| ---- | ---- | --- | :-: | :-: | :-: | :-: |")
    for i, (key, _) in enumerate(sorted(post_quals["overall_winner_count"].items(), key=lambda x: (filter(x[0]), -x[1]))[:limit]):
        print("|", "|".join([str(i + 1), str(key), division_to_team[key], percent(before["overall_winner_count"][key] / num_sims), percent(after["overall_winner_count"][key] / num_sims), percent(final["overall_winner_count"][key] / num_sims), percent(post_quals["overall_winner_count"][key] / num_sims)]), "|")
    print()
    
print("<h2>Chance of winning Einstein</h2>")
print()
    
print_einstein()

print("[details='By Division']")

for i in range(8):
    print("<h4>" + division_names[i] + "</h4>")
    print()
    print_einstein(filter=lambda x: 0 if x in divisions[i] else 1, limit=10)
    
print("[/details]")
print()
    
print("<h2>Most likely pairings</h2>")
print()

print("| Rank | ", pad("Division"), "|", pad("Team 1"), "|", pad("Team 2"), "|", pad("On Alliance"), "|", pad("Win Division"), "|",  pad("Win Einstein"), "|")
print("| --- | --- | --- | --- | :-: | :-: | :-: |")
for i, (key, _) in enumerate(sorted(post_quals["winning_pairs"].items(), key=lambda x: -x[1])[:25]):
    key_list = sorted(list(key), key=lambda x: -team_to_epa[x])
    print("|", "|".join([str(i + 1), division_to_team[key_list[0]], str(key_list[0]), str(key_list[1]), percent(post_quals["pairs"][key] / num_sims), percent(post_quals["einstein_pairs"][key] / num_sims), percent(post_quals["winning_pairs"][key] / num_sims)]))
    

<h2>Chance of winning Einstein</h2>

| Division |  Pre-Divisions Winner  |  Pre-Schedule Winner  |  Post-Schedule Winner  |  Post-Quals Winner  |
| --- | :-: | :-: | :-: | :-: |
| Archimedes|12.67%|51.32%|51.76%|56.09%|
| Curie|12.65%|3.79%|3.58%|8.05%|
| Daly|12.31%|2.56%|2.33%|2.71%|
| Galileo|12.7%|13.04%|14.86%|9.47%|
| Hopper|12.6%|12.25%|12.24%|15.14%|
| Johnson|12.55%|5.38%|4.65%|2.04%|
| Milstein|12.4%|8.1%|7.84%|3.89%|
| Newton|12.11%|3.57%|2.74%|2.61%|


<h2>Chance of Ranking High</h2>

| Rank |  Team&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Division&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Pre-Divisions Mean Rank  |  Pre-Schedule Mean Rank  |  Post-Schedule Mean Rank  |  Post-Quals Mean Rank  |
| --- | --- | --- | :-: | :-: | :-: | :-: |
| 1|2056|Archimedes|4.54|5.88|4.3|1.0 |
| 2|5940|Curie|10.09|8.64|10.04|1.0 |
| 3|5472|Daly|19.74|20.32|11.17|1.0 |
| 4|1678|Galileo|7.03|6.92|2.14|1.0 |
| 5|1323|Hopper|4.9|4.27|3.76|1.0 |
| 6|2075|Johnson|